In [1]:
!nvidia-smi

Sat Oct 14 15:48:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.23                 Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P8              18W / 290W |    759MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
from peft import PeftModel, PeftConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll
CUDA SETUP: CUDA runtime path found: C:\Users\Administrator\miniconda3\envs\GPU\bin\cudart64_12.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll...


In [4]:
peft_model_id = "Joo99/discord_chatbot"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                

In [9]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=126,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    decoded_answer = tokenizer.decode(gened[0])
    answer = decoded_answer.split("### 답변:")[1].split("<|끝|>")[0].split("|")[0].split("/")[0].split("\\")[0].strip()
    return answer


In [10]:
gen("안녕? 오늘 뭐해?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'너는 오늘 안 움직이니'

In [11]:
# __filename__ == 'test_discordbot.py'
import discord
from discord.ext import commands

discord_bot_token = ''
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix=">", intents=intents)

@bot.event
async def on_ready():
    print('Bot: {}'.format(bot.user))

@bot.event
async def on_message(message):

    if message.content.startswith('!'):
        req = message.content
        res = gen(req)
        await message.channel.send(res, tts=True)
        return

    await bot.process_commands(message)
bot.run(discord_bot_token)

[2023-10-14 15:53:49] [INFO    ] discord.client: logging in using static token
[2023-10-14 15:53:50] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 3ea89e689147d0d3b0904c4bd6e3cdb5).


Bot: 폰은정#3114


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
[2023-10-14 15:54:41] [WARNING ] discord.gateway: Shard ID None heartbeat blocked for more than 10 seconds.
Loop thread traceback (most recent call last):
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
    self.io_loop.start()
  File "c:\Users\Administrator\miniconda3\envs\GPU\lib\site-packages\tornado\platfor